In [ ]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt

from utils.detection.association_geodesic import squarize
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

from utils.physics.geodesic.distance import distance_point_point
from utils.physics.signal.make_spectrogram import make_spectrogram

plt.style.use('classic')
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 10,
    "axes.labelsize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
import math
from numpy.linalg import LinAlgError
import pandas as pd
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
from collections import Counter
from matplotlib.ticker import ScalarFormatter
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, MonthlyHomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [ ]:
CATALOG_PATH = "/media/plerolland/akoustik"
STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()

s = STATIONS.by_name("MAHY01")[0]
EQ_IMMERSION = 1303
EQ_BATHY = 2774
EQ_BEFORE = 1
EQ_BEFORE_PRECISE = -10
EQ_AFTER = 45
EQ_AFTER_PRECISE = 18
EQ_DATE = datetime.datetime(2020, 11, 22, 23, 12, 0)
EQ_DATE = EQ_DATE + 2*(EQ_DATE-s.date_start)*0.1293*1e-6
EQ_DIST_H = distance_point_point((-12.803144, 45.590244), s.get_pos())
EQ_DIST_V_H = (EQ_BATHY - EQ_IMMERSION)
EQ_DIST_V_S = 39421.875 - EQ_BATHY
print(EQ_DIST_H, EQ_DIST_V_H, EQ_DIST_V_S)

i_s = STATIONS.by_name("MAHY04")[0]
i_BEFORE = 10
i_AFTER = 10
i_DATE = datetime.datetime(2020, 10, 26, 13, 45, 10, 5*10**5)
i_DATE = i_DATE + 2*(i_DATE-s.date_start)*0.1293*1e-6

In [ ]:
m_EQ = s.get_manager()
data_EQ = m_EQ.get_segment(EQ_DATE - datetime.timedelta(seconds=EQ_BEFORE), EQ_DATE + datetime.timedelta(seconds=EQ_AFTER))
x_EQ = np.linspace(-EQ_BEFORE, EQ_AFTER, len(data_EQ))
data_EQ_PRECISE = m_EQ.get_segment(EQ_DATE - datetime.timedelta(seconds=EQ_BEFORE_PRECISE), EQ_DATE + datetime.timedelta(seconds=EQ_AFTER_PRECISE))
f_EQ, t_EQ, spectro_EQ = make_spectrogram(
    data_EQ, m_EQ.sampling_f, t_res=0.25, f_res=1,
    return_bins=True, normalize=True, vmin=60, vmax=120
)

m_i = i_s.get_manager()
data_i = m_i.get_segment(i_DATE - datetime.timedelta(seconds=i_BEFORE), i_DATE + datetime.timedelta(seconds=i_AFTER))
x_i = np.linspace(-i_BEFORE, i_AFTER, len(data_i))
f_i, t_i, spectro_i = make_spectrogram(
    data_i, m_i.sampling_f, t_res=0.25, f_res=1,
    return_bins=True, normalize=True, vmin=60, vmax=120
)

fig = plt.figure(figsize=(12, 6), facecolor='white')
width_in = 5.5
height_in = width_in / 1.08
fig.set_size_inches(width_in, height_in)

gs = gridspec.GridSpec(4, 2, height_ratios=[1, 1, 0.2, 1])
ax_spectro_EQ = fig.add_subplot(gs[0, 0])
ax_wave_EQ    = fig.add_subplot(gs[1, 0])
# blank line for space
ax_precise    = fig.add_subplot(gs[3, :])
ax_spectro_i = fig.add_subplot(gs[0, 1])
ax_wave_i = fig.add_subplot(gs[1, 1])

##### EQ

im0 = ax_spectro_EQ.imshow(
    spectro_EQ[::-1], cmap="inferno", aspect="auto", origin="lower",
    extent=(-EQ_BEFORE, EQ_AFTER, f_EQ[0], f_EQ[-1])
)
ax_spectro_EQ.set_facecolor('white')
ax_spectro_EQ.set_ylabel("Frequency (Hz)")
ax_spectro_EQ.set_xticklabels([])
ax_spectro_EQ.set_title("Earthquake-induced waves")
ax_spectro_EQ.set_xlim(x_EQ[0], x_EQ[-1])

ax_wave_EQ.plot(x_EQ, data_EQ, color='black')
ax_wave_EQ.set_facecolor('white')
ax_wave_EQ.set_xlim(x_EQ[0], x_EQ[-1])
ax_wave_EQ.set_ylim(-max(abs(data_EQ)),max(abs(data_EQ)))
ax_wave_EQ.set_ylabel(r"Amplitude ($\mu$Pa)")
ax_wave_EQ.set_xlabel(r"Time from event (s)")

x_EQ_PRECISE = np.linspace(-EQ_BEFORE_PRECISE, EQ_AFTER_PRECISE, len(data_EQ_PRECISE))
ax_precise.plot(x_EQ_PRECISE, data_EQ_PRECISE, color='black')
ax_precise.set_title("Earthquake-induced waves - zoom")
ax_precise.set_facecolor('white')
ax_precise.set_xlim(x_EQ_PRECISE[0], x_EQ_PRECISE[-1])
ax_precise.set_ylim(-max(abs(data_EQ_PRECISE)),max(abs(data_EQ_PRECISE)))
ax_precise.set_ylabel(r"Amplitude ($\mu$Pa)")
ax_precise.set_xlabel(r"Time from event (s)")

# origin
ORIGIN_TIME = [0]
P_TIMES = [10.88,12.51,14.63]
S_TIMES = [16.44]
T_TIMES = [np.sqrt(EQ_DIST_V_S ** 2 + (EQ_DIST_H / 2) ** 2) / 5000 + np.sqrt(EQ_DIST_V_H ** 2 + (EQ_DIST_H / 2) ** 2) / 1500,
           EQ_DIST_V_S / 5000 + np.sqrt(EQ_DIST_V_H ** 2 + EQ_DIST_H ** 2) / 1500]
to_plot = {"origin": ("black",ORIGIN_TIME),
           "P": ("red",P_TIMES),
           "S": ("green",S_TIMES),
           "T": ("royalblue",T_TIMES),
           }
for name, (color, times) in to_plot.items():
    if name == "T" and len(times) == 2:
        t_min, t_max = times
        for ax in [ax_spectro_EQ, ax_wave_EQ, ax_precise]:
            ax.axvspan(t_min, t_max, color=color, alpha=0.3, zorder=1, label="T interval")
    else:
        for time in times:
            ax_spectro_EQ.axvline(time, color=color, linestyle='--', linewidth=1.5, zorder=2)
            ax_wave_EQ.axvline(time, color=color, linestyle='--', linewidth=1.5, zorder=2)
            ax_precise.axvline(time, color=color, linestyle='--', linewidth=1.5, zorder=2)


##### i

im1 = ax_spectro_i.imshow(
    spectro_i[::-1], cmap="inferno", aspect="auto", origin="lower",
    extent=(-i_BEFORE, i_AFTER, f_EQ[0], f_EQ[-1])
)
ax_spectro_i.set_xticklabels([])
ax_spectro_i.set_title("H-wave")
ax_spectro_i.set_xlim(x_i[0], x_i[-1])

x_i = np.linspace(-i_BEFORE, i_AFTER, len(data_i))
ax_wave_i.plot(x_i, data_i, color='black')
ax_wave_i.set_xlim(x_i[0], x_i[-1])
ax_wave_i.set_ylim(-1*max(abs(data_i)),1*max(abs(data_i)))
ax_wave_i.set_xlabel(r"Time from signal (s)")
formatter = ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
ax_wave_i.yaxis.set_major_formatter(formatter)

fig.patch.set_facecolor('xkcd:white')
plt.savefig(
    f'../../../../data/MAHY/figures/signal_examples.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
print(P := np.sqrt(EQ_DIST_H**2+EQ_DIST_V_S**2) / (to_plot["P"][1][0] - EQ_DIST_V_H/1500))
print(S := np.sqrt(EQ_DIST_H**2+EQ_DIST_V_S**2) / (to_plot["S"][1][0] - EQ_DIST_V_H/1500))
print(P/S)